# Exemplo de xanelas con watermark
Como se viu no notebook anterior, para facer operacións de agregación sobre xanelas é necesario empregar *watermarking*. Neste exemplo imos repetir o caso anterior pero usando unha marca de auga, neste caso de 30 minutos.


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
import string

spark = SparkSession.builder \
    .appName("xanelas-4") \
    .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
    .getOrCreate()

print("Versión: ",spark.version)

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/hadoop/.ivy2/cache
The jars for the packages stored in: /home/hadoop/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.kafka#kafka-clients added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-7636a345-8a8f-4ead-b5f8-d8dcff36dac2;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.1.0 in central
	found io.delta#delta-storage;3.1.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.7 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found org.xerial.snappy#snappy-java;1.1.10.5 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.fi

Versión:  3.5.7


O esquema é o mesmo que no caso anterior:

In [2]:
# Definimos el esquema de los datos de entrada
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
bolsaSchema = StructType([
    StructField("CreatedTime", StringType()),
    StructField("Type", StringType()),
    StructField("Amount", IntegerType()),
    StructField("BrokerCode", StringType())
])

O fluxo de entrada é o mesmo:

In [3]:
# Configuramos la lectura de fichero en formato JSON
rawDF = spark.readStream \
        .format("json") \
        .option("path", "/data/bolsa") \
        .option("maxFilesPerTrigger", 1) \
        .schema(bolsaSchema) \
        .load()

rawDF.printSchema()

root
 |-- CreatedTime: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Amount: integer (nullable = true)
 |-- BrokerCode: string (nullable = true)



O DF refinado tamén é o mesmo:

In [4]:
from pyspark.sql.functions import to_timestamp, col, expr
accionesDF = rawDF.withColumn("CreatedTime", to_timestamp(col("CreatedTime"), "yyyy-MM-dd HH:mm:ss")) \
    .withColumn("Compras", expr("case when Type == 'BUY' then Amount else 0 end")) \
    .withColumn("Ventas", expr("case when Type == 'SELL' then Amount else 0 end"))

accionesDF.printSchema()

root
 |-- CreatedTime: timestamp (nullable = true)
 |-- Type: string (nullable = true)
 |-- Amount: integer (nullable = true)
 |-- BrokerCode: string (nullable = true)
 |-- Compras: integer (nullable = true)
 |-- Ventas: integer (nullable = true)



Engadimos a marca de auga mediante a función **withWatermark**, indicando a columna á que se aplica e o limiar de tempo. Agora empregaremos un *sink* de tipo ficheiro usando o modo *append*. Iso si, hai que ter en conta que **os datos se escribirán unha vez transcorrido o limiar establecido na marca de auga**.


In [5]:
from pyspark.sql.functions import window, sum

# Agrupamos as operacións por xanelas de 15 minutos e calculamos agregacións.
# Ao tratarse de agregacións sobre xanelas en streaming, engádese unha marca de auga (watermark)
# para manexar eventos atrasados e permitir o modo de saída 'append'.
windowDF = accionesDF \
    .withWatermark("CreatedTime", "30 minutes") \  # Acepta eventos atrasados ata 30 minutos.
    .groupBy(
         window(col("CreatedTime"), "15 minutes")   # Define xanelas temporais fixas de 15 minutos.
    ) \
    .agg(
         sum("Compras").alias("Compras"),           # Suma do importe de compras na xanela.
         sum("Ventas").alias("Ventas")              # Suma do importe de vendas na xanela.
    )

# Seleccionamos e aplanamos o struct 'window' en dúas columnas (inicio e fin da xanela)
# para facilitar a saída.
salidaDF = windowDF.select("window.start", "window.end", "Compras", "Ventas")

# Configuramos a escrita en streaming:
# - Formato Parquet
# - Modo append (require watermark + agregación con xanela)
# - Ruta de saída e checkpoint (obrigatorio para tolerancia a fallos e estado)
# - Trigger cada 1 minuto
bolsaWriterQuery = salidaDF.writeStream \
    .format("parquet") \
    .queryName("BolsaWQuery") \
    .outputMode("append") \
    .option("path", "/salida_bolsa") \
    .option("checkpointLocation", "/checkpoints/chk-point-dir-caso7") \
    .trigger(processingTime="1 minute") \
    .start()


In [12]:
rawBolsaDF = spark.read \
    .format("parquet") \
    .option("path", "/salida_bolsa") \
    .load()
rawBolsaDF.show()

+-------------------+-------------------+-------+------+
|              start|                end|Compras|Ventas|
+-------------------+-------------------+-------+------+
|2022-05-09 10:00:00|2022-05-09 10:15:00|    800|     0|
|2022-05-09 10:45:00|2022-05-09 11:00:00|      0|   700|
|2022-05-09 10:30:00|2022-05-09 10:45:00|    900|     0|
|2022-05-09 10:15:00|2022-05-09 10:30:00|    800|   400|
+-------------------+-------------------+-------+------+



A continuación calcúlase o acumulado total de compras e vendas ao longo do tempo empregando funcións de xanela. Ademais, obtense o resultado neto como a diferenza entre ambos valores.


In [13]:
from pyspark.sql import Window

# Definimos unha xanela (Window spec) para calcular acumulados:
# - Ordenamos polas marcas temporais de fin de xanela ('end')
# - O marco de filas vai desde o inicio (unboundedPreceding) ata a fila actual (currentRow),
#   é dicir, un acumulado progresivo.
ventanaTotal = Window.orderBy("end") \
    .rowsBetween(Window.unboundedPreceding, Window.currentRow)

# A partir do DF con resultados por xanela (rawBolsaDF), calculamos:
# - Compras acumuladas ata cada fila
# - Vendas acumuladas ata cada fila
# - Neto acumulado como diferenza entre Compras e Vendas
salidaDF = rawBolsaDF \
    .withColumn("Compras", sum("Compras").over(ventanaTotal)) \
    .withColumn("Ventas", sum("Ventas").over(ventanaTotal)) \
    .withColumn("Neto", expr("Compras - Ventas"))

# Mostramos o resultado completo sen truncar columnas, para ver valores e datas enteiras.
salidaDF.show(truncate=False)


+-------------------+-------------------+-------+------+----+
|start              |end                |Compras|Ventas|Neto|
+-------------------+-------------------+-------+------+----+
|2022-05-09 10:00:00|2022-05-09 10:15:00|800    |0     |800 |
|2022-05-09 10:15:00|2022-05-09 10:30:00|1600   |400   |1200|
|2022-05-09 10:30:00|2022-05-09 10:45:00|2500   |400   |2100|
|2022-05-09 10:45:00|2022-05-09 11:00:00|2500   |1100  |1400|
+-------------------+-------------------+-------+------+----+

